## 4.2 迁移学习实战

### 1迁移学习的基本原理
迁移学习是用已有模型来训练新模型，这样做的目的是省去过多的训练任务，比如猫狗分类问题，如果我们有很多狗的标注图片，我们可以先用狗训练模型，然后在全连接层用少量毛的图片就可以训练出一个可以选出猫的模型，但是这种做法的前提是这两样东西比较类似才行，如果用人的图片先训练肯定不行。

### 2数据预处理
原始数据可能存在一些非常大或非常小的值，为了让数据容易被学习，我们会把数据限制在一个范围内，这里使用"0均值归一化处理"，它的原理就是把数据转化为均值为0，方差为1的数据。

In [1]:
import torch

In [2]:
mean = torch.Tensor([16, 48, 27, 238, 12]).mean()
mean

tensor(68.2000)

In [3]:
std = torch.Tensor([16, 48, 27, 238, 12]).std()
std

tensor(95.9437)

归一化我们使用公式:
$$
x'={x-\mu \over \sigma}
$$
其中$\mu$是均值$\sigma$是标准差

In [4]:
#计算均值
normalized = torch.Tensor([16, 48, 27, 238, 12]).add(-68.2).div(95.9437335108448)
normalized

tensor([-0.5441, -0.2105, -0.4294,  1.7698, -0.5858])

In [5]:
print(normalized.mean())
print(normalized.std())

tensor(2.3842e-08)
tensor(1.)


### 3准备数据集
网络上找200张羊驼和熊猫照片作为训练集，另外收集100张照片作为测试集

### 4ImageFolder与预处理
预处理主要使用到transforms.Compose()进行预处理，datasets.ImageFolder()选出数据文件夹，torch.utils.data.DataLoader()进行加载数据。

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import os

In [7]:
data_transforms = {
    'train': transforms.Compose([
        #transforms.Scale(230),
        transforms.Grayscale(230),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
    'test': transforms.Compose([
        #transforms.Scale(230),
        transforms.Grayscale(230),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
}

In [8]:
data_directory = 'data'
trainset = datasets.ImageFolder(os.path.join(data_directory, 'train'), data_transforms['train'])
testset = datasets.ImageFolder(os.path.join(data_directory, 'test'), data_transforms['test'])

In [9]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=5, shuffle=True, num_workers=4)
testloader = torch.utils.data.DataLoader(testset, batch_size=5, shuffle=True, num_workers=4)

### 5加载预训练模型
我们可以挑选已经预训练好的模型进行迁移学习


In [10]:
alexnet = models.alexnet(pretrained=True)
print(alexnet)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [11]:
for param in alexnet.parameters():
    param.requires_grad = False

alexnet.classifier = nn.Sequential(
    nn.Dropout(),
    nn.Linear(256 * 6 * 6, 4096),
    nn.ReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(4096, 4096),
    nn.ReLU(inplace=True),
    nn.Linear(4096, 2), )

### 6定义训练与测试函数

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(alexnet.classifier.parameters(), lr=0.001, momentum=0.9)

In [13]:
def train(model, criterion, optimizer, epochs=1):
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 10 == 9:
                print('[Epoch:%d, Batch:%5d] Loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
                running_loss = 0.0

    print('Finished Training')

In [14]:
def test(testloader, model):
    correct = 0
    total = 0
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
    print('Accuracy on the test set: %d %%' % (100 * correct / total))

### 7训练与测试结果

In [15]:
def load_param(model, path):
    if os.path.exists(path):
        model.load_state_dict(torch.load(path))


def save_param(model, path):
    torch.save(model.state_dict(), path)

In [16]:
load_param(alexnet, 'tl_model.pkl')

train(alexnet, criterion, optimizer, epochs=2)

save_param(alexnet, 'tl_model.pkl')

test(testloader, alexnet)

ValueError: Caught ValueError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/Users/limuyuan/Documents/Introduction-to-pytorch-deep-learning/venv/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/Users/limuyuan/Documents/Introduction-to-pytorch-deep-learning/venv/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/Users/limuyuan/Documents/Introduction-to-pytorch-deep-learning/venv/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/Users/limuyuan/Documents/Introduction-to-pytorch-deep-learning/venv/lib/python3.8/site-packages/torchvision/datasets/folder.py", line 232, in __getitem__
    sample = self.transform(sample)
  File "/Users/limuyuan/Documents/Introductio